In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import random

import pandas as pd

file_path = "../../web_service_dataset.csv"
df = pd.read_csv(file_path)


In [2]:
df

,src_ip,src_port,dst_ip,dst_port,proto,pktTotalCount,octetTotalCount,min_ps,max_ps,avg_ps,...,b_std_dev_ps,b_flowStart,b_flowEnd,b_flowDuration,b_min_piat,b_max_piat,b_avg_piat,b_std_dev_piat,category,web_service
0,192.168.121.1,67,172.16.255.185,67,UDP,22,7620,328,394,346.363636,...,2.000000,1.555955e+09,1.555955e+09,1.554399e+12,1.035380e-02,198.657965,51.689181,84.916348,Network,DHCP
1,192.168.121.1,67,172.16.255.186,67,UDP,17,5670,328,354,333.529412,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,Network,DHCP
2,192.168.121.1,67,172.16.255.185,67,UDP,43,15124,328,394,351.720930,...,1.332347,1.555964e+09,1.555965e+09,1.554409e+12,2.233195e-02,340.268454,89.927588,124.270745,Network,DHCP
3,192.168.121.1,67,172.16.255.186,67,UDP,30,10086,328,352,336.200000,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,Network,DHCP
4,192.168.121.1,0,10.130.1.166,0,ICMP,1,56,56,56,56.000000,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,Network,ICMP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2704834,192.168.128.252,137,172.16.255.186,137,UDP,9,702,78,78,78.000000,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,System,NetBIOS
2704835,192.168.128.252,138,172.16.255.185,138,UDP,3,687,229,229,229.000000,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,System,NetBIOS
2704836,192.168.128.252,138,172.16.255.186,138,UDP,3,687,229,229,229.000000,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,System,NetBIOS
2704837,192.168.128.252,49879,108.177.11.188,5228,TCP,75,3520,40,66,46.933333,...,4.228907,1.559770e+09,1.559771e+09,1.558211e+12,1.150203e-02,90.155711,44.999017,14.822745,Web,Google


In [3]:
# 4.1
# Group by the specified columns
grouped = df.groupby(['src_ip', 'src_port', 'dst_ip', 'dst_port', 'web_service'])

# Count the number of flows in each group
group_counts = grouped.size()

# Filter out groups with fewer than 10 flows
filtered_groups = group_counts[group_counts >= 10]

# Count the remaining groups
num_remaining_groups = filtered_groups.shape[0]

print(f"Total number of groups: {group_counts.shape[0]}")
print(f"Number of groups with at least 10 flows: {num_remaining_groups}")

Total number of groups: 2433314
Number of groups with at least 10 flows: 2057


In [4]:
# 4.2
# Step 1: Filter out classes with less than 400 flows
web_service_counts = df['web_service'].value_counts()
valid_web_services = web_service_counts[web_service_counts >= 400].index
df_filtered = df[df['web_service'].isin(valid_web_services)]

# Step 2: Segment each sub-dataset into chunks of 10 flows
chunks = []
for web_service in valid_web_services:
    sub_df = df_filtered[df_filtered['web_service'] == web_service]
    num_chunks = len(sub_df) // 10
    for i in range(num_chunks):
        chunk = sub_df.iloc[i*10:(i+1)*10]
        chunks.append(chunk)

# Step 3: Remove redundant information and keep only useful features
# Assuming useful features are 'src_ip', 'dst_ip', 'src_port', 'dst_port', 'pktTotalCount', 'octetTotalCount', 'category', 'web_service'
useful_features = ['src_ip', 'dst_ip', 'src_port', 'dst_port', 'pktTotalCount', 'octetTotalCount', 'category', 'web_service']
chunks = [chunk[useful_features] for chunk in chunks]

# Step 4: Count the number of chunks and classes
num_chunks = len(chunks)
num_classes = len(valid_web_services)

# Step 5: Check if the sample quantities of all classes are balanced
class_counts = df_filtered['web_service'].value_counts()
balanced = class_counts.min() == class_counts.max()

print(f"Number of chunks (samples): {num_chunks}")
print(f"Number of classes: {num_classes}")
print(f"Are the sample quantities of all classes balanced? {'Yes' if balanced else 'No'}")


KeyboardInterrupt

